# Tesla Stock Hyperparameter Tuning Project

In this project we are going to analyze what would the highest point of the Tesla's stock would be in any given day, we would be implementing `l1`, `l2` depending of the accuracy and alongside we would be finding the best `C` hyperparameter for either regularization.

### Importing the data and useful libraries

In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Now let's import our dataset and define our 'X' and 'y' variables
data = pd.read_csv('TESLA.csv')

# Let's standardized all our columns

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_features = scaler.fit_transform(data[['Open', 'Low', 'Close', 'Adj Close', 'Volume']])
scaled_df = pd.DataFrame(scaled_features, columns=['Open', 'Low', 'Close', 'Adj Close', 'Volume'])


X = scaled_df
y = data['High']

scaled_df.head()

,Open,Low,Close,Adj Close,Volume
0,-0.747394,-0.748661,-0.744583,-0.744583,2.373689
1,-0.743101,-0.744929,-0.744621,-0.744621,2.069460
2,-0.743601,-0.746892,-0.745804,-0.745804,0.341740
3,-0.744865,-0.747903,-0.747550,-0.747550,-0.251422
4,-0.746762,-0.749768,-0.749504,-0.749504,0.081300


In [34]:
# Checking Nan Values
print(data.describe())
data.isna()

        Unnamed: 0         Open         High          Low        Close  \
count  3637.000000  3637.000000  3637.000000  3637.000000  3637.000000   
mean   1818.000000    80.080574    81.832710    78.219276    80.067952   
std    1050.055792   105.466131   107.807170   102.931208   105.409448   
min       0.000000     1.076000     1.108667     0.998667     1.053333   
25%     909.000000    12.046667    12.320000    11.746667    12.065333   
50%    1818.000000    17.833332    18.080000    17.563999    17.846666   
75%    2727.000000   176.070007   179.770004   173.169998   176.880005   
max    3636.000000   411.470001   414.496674   405.666656   409.970001   

         Adj Close        Volume  
count  3637.000000  3.637000e+03  
mean     80.067952  9.667330e+07  
std     105.409448  7.787314e+07  
min       1.053333  1.777500e+06  
25%      12.065333  4.868250e+07  
50%      17.846666  8.198100e+07  
75%     176.880005  1.223940e+08  
max     409.970001  9.140820e+08  


,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
3632,False,False,False,False,False,False,False,False
3633,False,False,False,False,False,False,False,False
3634,False,False,False,False,False,False,False,False
3635,False,False,False,False,False,False,False,False


As we could see there is no `NaN` values on our data set so we can proceed with our analisis

In [35]:
data.dtypes

Unnamed: 0      int64
Date           object
Open          float64
High          float64
Low           float64
Close         float64
Adj Close     float64
Volume          int64
dtype: object

## Splitting our data

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Now we're going to work on a SVR model, so we're first going to import the library to then fit our model with a `C` value of `1`

In [40]:
from sklearn.svm import SVR

svr = SVR(kernel='rbf', C=1.0, epsilon=0.1)

# Trainig our model
svr.fit(X_train, y_train)

# predicted on Test data
y_pred = svr.predict(X_test)

Now we're going to test our SVR mode with our initial parameter (without tuning) and we compare them later on with our hypertuned ones

In [41]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R² Score: {r2}")


RMSE: 19.295437654851547
MAE: 5.594503301118142
R² Score: 0.9678073622249379


## Hyperparameter Tuning

In [43]:
param_grid = {
    'kernel': ['linear', 'poly', 'rbf'],
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 0.5, 1],
    'gamma': ['scale', 'auto']
}

grid_search = GridSearchCV(svr, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_


Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] END ....C=0.1, epsilon=0.01, gamma=scale, kernel=linear; total time=   0.1s
[CV] END ....C=0.1, epsilon=0.01, gamma=scale, kernel=linear; total time=   0.1s
[CV] END ....C=0.1, epsilon=0.01, gamma=scale, kernel=linear; total time=   0.2s
[CV] END ....C=0.1, epsilon=0.01, gamma=scale, kernel=linear; total time=   0.2s
[CV] END ....C=0.1, epsilon=0.01, gamma=scale, kernel=linear; total time=   0.1s
[CV] END ......C=0.1, epsilon=0.01, gamma=scale, kernel=poly; total time=   0.2s
[CV] END ......C=0.1, epsilon=0.01, gamma=scale, kernel=poly; total time=   0.1s
[CV] END ......C=0.1, epsilon=0.01, gamma=scale, kernel=poly; total time=   0.2s
[CV] END ......C=0.1, epsilon=0.01, gamma=scale, kernel=poly; total time=   0.2s
[CV] END ......C=0.1, epsilon=0.01, gamma=scale, kernel=poly; total time=   0.2s
[CV] END .......C=0.1, epsilon=0.01, gamma=scale, kernel=rbf; total time=   0.3s
[CV] END .......C=0.1, epsilon=0.01, gamma=scal

[CV] END ..........C=0.1, epsilon=1, gamma=scale, kernel=rbf; total time=   0.2s
[CV] END ..........C=0.1, epsilon=1, gamma=scale, kernel=rbf; total time=   0.2s
[CV] END ..........C=0.1, epsilon=1, gamma=scale, kernel=rbf; total time=   0.3s
[CV] END ..........C=0.1, epsilon=1, gamma=scale, kernel=rbf; total time=   0.2s
[CV] END ........C=0.1, epsilon=1, gamma=auto, kernel=linear; total time=   0.0s
[CV] END ........C=0.1, epsilon=1, gamma=auto, kernel=linear; total time=   0.0s
[CV] END ........C=0.1, epsilon=1, gamma=auto, kernel=linear; total time=   0.0s
[CV] END ........C=0.1, epsilon=1, gamma=auto, kernel=linear; total time=   0.0s
[CV] END ........C=0.1, epsilon=1, gamma=auto, kernel=linear; total time=   0.0s
[CV] END ..........C=0.1, epsilon=1, gamma=auto, kernel=poly; total time=   0.1s
[CV] END ..........C=0.1, epsilon=1, gamma=auto, kernel=poly; total time=   0.2s
[CV] END ..........C=0.1, epsilon=1, gamma=auto, kernel=poly; total time=   0.2s
[CV] END ..........C=0.1, ep

[CV] END ..........C=1, epsilon=0.5, gamma=auto, kernel=poly; total time=   0.1s
[CV] END ..........C=1, epsilon=0.5, gamma=auto, kernel=poly; total time=   0.1s
[CV] END ...........C=1, epsilon=0.5, gamma=auto, kernel=rbf; total time=   0.2s
[CV] END ...........C=1, epsilon=0.5, gamma=auto, kernel=rbf; total time=   0.2s
[CV] END ...........C=1, epsilon=0.5, gamma=auto, kernel=rbf; total time=   0.2s
[CV] END ...........C=1, epsilon=0.5, gamma=auto, kernel=rbf; total time=   0.2s
[CV] END ...........C=1, epsilon=0.5, gamma=auto, kernel=rbf; total time=   0.2s
[CV] END .........C=1, epsilon=1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END .........C=1, epsilon=1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END .........C=1, epsilon=1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END .........C=1, epsilon=1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END .........C=1, epsilon=1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END ...........C=1, eps

[CV] END ........C=10, epsilon=0.5, gamma=scale, kernel=poly; total time=   0.2s
[CV] END ........C=10, epsilon=0.5, gamma=scale, kernel=poly; total time=   0.2s
[CV] END ........C=10, epsilon=0.5, gamma=scale, kernel=poly; total time=   0.2s
[CV] END ........C=10, epsilon=0.5, gamma=scale, kernel=poly; total time=   0.2s
[CV] END ........C=10, epsilon=0.5, gamma=scale, kernel=poly; total time=   0.2s
[CV] END .........C=10, epsilon=0.5, gamma=scale, kernel=rbf; total time=   0.1s
[CV] END .........C=10, epsilon=0.5, gamma=scale, kernel=rbf; total time=   0.1s
[CV] END .........C=10, epsilon=0.5, gamma=scale, kernel=rbf; total time=   0.1s
[CV] END .........C=10, epsilon=0.5, gamma=scale, kernel=rbf; total time=   0.1s
[CV] END .........C=10, epsilon=0.5, gamma=scale, kernel=rbf; total time=   0.1s
[CV] END .......C=10, epsilon=0.5, gamma=auto, kernel=linear; total time=   0.3s
[CV] END .......C=10, epsilon=0.5, gamma=auto, kernel=linear; total time=   0.2s
[CV] END .......C=10, epsilo

[CV] END ......C=100, epsilon=0.1, gamma=auto, kernel=linear; total time=   2.7s
[CV] END ......C=100, epsilon=0.1, gamma=auto, kernel=linear; total time=   2.3s
[CV] END ......C=100, epsilon=0.1, gamma=auto, kernel=linear; total time=   3.1s
[CV] END ........C=100, epsilon=0.1, gamma=auto, kernel=poly; total time=   0.6s
[CV] END ........C=100, epsilon=0.1, gamma=auto, kernel=poly; total time=   0.6s
[CV] END ........C=100, epsilon=0.1, gamma=auto, kernel=poly; total time=   0.8s
[CV] END ........C=100, epsilon=0.1, gamma=auto, kernel=poly; total time=   0.7s
[CV] END ........C=100, epsilon=0.1, gamma=auto, kernel=poly; total time=   0.7s
[CV] END .........C=100, epsilon=0.1, gamma=auto, kernel=rbf; total time=   0.4s
[CV] END .........C=100, epsilon=0.1, gamma=auto, kernel=rbf; total time=   0.3s
[CV] END .........C=100, epsilon=0.1, gamma=auto, kernel=rbf; total time=   0.3s
[CV] END .........C=100, epsilon=0.1, gamma=auto, kernel=rbf; total time=   0.4s
[CV] END .........C=100, eps

Using the best model for prediction and evaluation

In [44]:
y_pred_best = best_model.predict(X_test)

rmse_best = mean_squared_error(y_test, y_pred_best, squared=False)
print(f"Tuned RMSE: {rmse_best}")


Tuned RMSE: 1.3601995516608507


In [45]:
best_model.score(X_test, y_test)

0.9998400248409719